In [1]:
import psycopg2
from datetime import datetime
import os
from dotenv import load_dotenv

# Actually call the load_dotenv() function
load_dotenv()

# ============================================================================
# CONFIGURATION - Please fill in these values
# ============================================================================
# AWS RDS PostgreSQL connection parameters
DB_HOST = os.getenv("DB_HOST", "your-rds-endpoint.region.rds.amazonaws.com")
DB_PORT = os.getenv("DB_PORT", "5432")
DB_NAME = os.getenv("DB_NAME", "your_database_name")
DB_USER = os.getenv("DB_USER", "your_username")
DB_PASSWORD = os.getenv("DB_PASSWORD", "My#22040two")

print("Configuration loaded. Please ensure all connection parameters are set correctly.")


Configuration loaded. Please ensure all connection parameters are set correctly.


In [2]:
print(DB_PASSWORD)

My#22040two


In [3]:
# ============================================================================
# CONNECT TO POSTGRESQL DATABASE
# ============================================================================
try:
    conn = psycopg2.connect(
        host=DB_HOST,
        port=DB_PORT,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD
    )
    conn.autocommit = False
    cursor = conn.cursor()
    print("✓ Successfully connected to PostgreSQL database")
except Exception as e:
    print(f"✗ Error connecting to database: {e}")
    raise


✓ Successfully connected to PostgreSQL database


### Users


In [3]:
# ============================================================================
# CREATE USERS TABLE
# ============================================================================
create_users_table_query = """
CREATE TABLE IF NOT EXISTS users (
    id SERIAL PRIMARY KEY,
    email VARCHAR(255) UNIQUE NOT NULL,
    password_hash VARCHAR(255) NOT NULL,
    full_name VARCHAR(255),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""

try:
    cursor.execute(create_users_table_query)
    conn.commit()
    print("✓ Table 'users' created successfully")
except Exception as e:
    conn.rollback()
    print(f"✗ Error creating table: {e}")
    raise


✓ Table 'users' created successfully


In [8]:
# ============================================================================
# QUERY AND PRINT ALL ROWS FROM USERS TABLE
# ============================================================================
try:
    cursor.execute("SELECT id, email, full_name, created_at, updated_at FROM users;")
    users = cursor.fetchall()
    if not users:
        print("No users found in the 'users' table.")
    else:
        print("All users in the 'users' table:")
        for user in users:
            print({
                "id": user[0],
                "email": user[1],
                "full_name": user[2],
                "created_at": user[3],
                "updated_at": user[4]
            })
except Exception as e:
    print(f"✗ Error querying users table: {e}")
    raise


All users in the 'users' table:
{'id': 1, 'email': 'mukesh.yerra@gmail.com', 'full_name': 'Mukesh Y', 'created_at': datetime.datetime(2025, 11, 21, 1, 18, 32, 345108), 'updated_at': datetime.datetime(2025, 11, 21, 1, 18, 32, 345108)}
{'id': 2, 'email': 'max@gmail.com', 'full_name': 'max', 'created_at': datetime.datetime(2025, 11, 21, 1, 26, 7, 506045), 'updated_at': datetime.datetime(2025, 11, 21, 1, 26, 7, 506045)}


### Convo History

In [4]:
# ============================================================================
# CREATE CONVERSATION_HISTORY TABLE
# ============================================================================
create_table_query = """
CREATE TABLE IF NOT EXISTS conversation_history (
    id SERIAL PRIMARY KEY,
    user_id INTEGER NOT NULL,
    timestamp TIMESTAMP NOT NULL,
    user_query TEXT NOT NULL,
    llm_response TEXT NOT NULL
);
"""

try:
    cursor.execute(create_table_query)
    conn.commit()
    print("✓ Table 'conversation_history' created successfully")
except Exception as e:
    conn.rollback()
    print(f"✗ Error creating table: {e}")
    raise


✓ Table 'conversation_history' created successfully


In [5]:
# ============================================================================
# POPULATE TABLE WITH SAMPLE CONVERSATIONS
# ============================================================================
# Sample conversations for user_id = 1
import time
sample_conversations = [
    {
        "user_id": 1,
        "timestamp": datetime.now(),
        "user_query": "What is the main topic discussed in the PDF?",
        "llm_response": "The PDF discusses machine learning fundamentals, including supervised and unsupervised learning approaches, neural networks, and their applications in various domains."
    },
    {
        "user_id": 1,
        "timestamp": datetime.now(),
        "user_query": "Can you summarize the key findings?",
        "llm_response": "The key findings include: 1) Neural networks show superior performance on complex datasets, 2) Transfer learning significantly reduces training time, 3) Data preprocessing is crucial for model accuracy."
    },
    {
        "user_id": 1,
        "timestamp": datetime.now(),
        "user_query": "What are the limitations mentioned?",
        "llm_response": "The main limitations discussed are: computational resource requirements, need for large labeled datasets, potential overfitting with small datasets, and interpretability challenges with deep learning models."
    },
    {
        "user_id": 1,
        "timestamp": datetime.now(),
        "user_query": "How does this compare to traditional methods?",
        "llm_response": "Compared to traditional methods, the approach shows 15-20% improvement in accuracy, but requires 3x more computational resources. The trade-off is generally favorable for large-scale applications."
    },
    {
        "user_id": 1,
        "timestamp": datetime.now(),
        "user_query": "What are the recommended next steps?",
        "llm_response": "The recommended next steps include: expanding the dataset, experimenting with different architectures, implementing ensemble methods, and conducting real-world validation studies."
    }
]

# Insert sample conversations
insert_query = """
INSERT INTO conversation_history (user_id, timestamp, user_query, llm_response)
VALUES (%s, %s, %s, %s)
"""

try:
    for conv in sample_conversations:
        cursor.execute(
            insert_query,
            (conv["user_id"], conv["timestamp"], conv["user_query"], conv["llm_response"])
        )
        time.sleep(1)
    
    conn.commit()
    print(f"✓ Successfully inserted {len(sample_conversations)} sample conversations")
except Exception as e:
    conn.rollback()
    print(f"✗ Error inserting data: {e}")
    raise


✓ Successfully inserted 5 sample conversations


In [10]:
# ============================================================================
# VERIFY DATA INSERTION
# ============================================================================
verify_query = """
SELECT id, user_id, timestamp, user_query, llm_response
FROM conversation_history
WHERE user_id = 2
ORDER BY timestamp DESC;
"""

try:
    cursor.execute(verify_query)
    results = cursor.fetchall()
    
    print(f"\n✓ Found {len(results)} conversations for user_id = 1:\n")
    for row in results:
        print(f"ID: {row[0]}")
        print(f"User ID: {row[1]}")
        print(f"Timestamp: {row[2]}")
        print(f"User Query: {row[3][:80]}..." if len(row[3]) > 80 else f"User Query: {row[3]}")
        print(f"LLM Response: {row[4][:80]}..." if len(row[4]) > 80 else f"LLM Response: {row[4]}")
        print("-" * 80)
except Exception as e:
    print(f"✗ Error verifying data: {e}")



✓ Found 2 conversations for user_id = 1:

ID: 24
User ID: 2
Timestamp: 2025-11-21 10:30:51.667100
User Query: can you tell me what we discussed earlier ?
LLM Response: Earlier, you greeted me with "hi," and I responded by asking how I could assist ...
--------------------------------------------------------------------------------
ID: 23
User ID: 2
Timestamp: 2025-11-21 10:30:35.152263
User Query: hi
LLM Response: Hello! How can I assist you today?
--------------------------------------------------------------------------------


In [ ]:
# ============================================================================
# CLOSE CONNECTION
# ============================================================================
cursor.close()
conn.close()
print("\n✓ Database connection closed successfully")


### Logs


In [4]:
# ============================================================================
# CREATE LOGS TABLE
# ============================================================================
create_logs_table_query = """
CREATE TABLE IF NOT EXISTS logs (
    id SERIAL PRIMARY KEY,
    u_id INTEGER NOT NULL,
    timestamp TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
    query TEXT,
    processed_query TEXT,
    context TEXT,
    past_memory TEXT,
    llm_response TEXT
);
"""

try:
    cursor.execute(create_logs_table_query)
    conn.commit()
    print("✓ 'logs' table created (if not exists).")
except Exception as e:
    print(f"✗ Error creating 'logs' table: {e}")
    conn.rollback()


✓ 'logs' table created (if not exists).


In [5]:
# ============================================================================
# INSERT DUMMY DATA INTO LOGS TABLE AND QUERY IT
# ============================================================================

# Insert dummy data
insert_logs_query = """
INSERT INTO logs (u_id, query, processed_query, context, past_memory, llm_response)
VALUES (%s, %s, %s, %s, %s, %s)
"""
dummy_data = [
    (1, "What is the capital of France?", "capital of France", "No context", "[]", "Paris is the capital of France."),
    (2, "List the planets in the solar system.", "planets in solar system", "Astronomy", "[]", "The planets are Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune."),
    (3, "Translate 'Hello' to Spanish.", "translate Hello Spanish", "Language", "[]", "'Hello' in Spanish is 'Hola'.")
]

try:
    for row in dummy_data:
        cursor.execute(insert_logs_query, row)
    conn.commit()
    print("✓ Dummy logs inserted successfully.")
except Exception as e:
    print(f"✗ Error inserting dummy logs: {e}")
    conn.rollback()

# Query the logs table
print("\nLogs Table Contents:")
try:
    cursor.execute("SELECT id, u_id, timestamp, query, processed_query, context, past_memory, llm_response FROM logs")
    records = cursor.fetchall()
    for record in records:
        print(record)
except Exception as e:
    print(f"✗ Error querying logs table: {e}")


✓ Dummy logs inserted successfully.

Logs Table Contents:
(1, 1, datetime.datetime(2025, 11, 20, 1, 0, 29, 245638), 'What is the capital of France?', 'capital of France', 'No context', '[]', 'Paris is the capital of France.')
(2, 2, datetime.datetime(2025, 11, 20, 1, 0, 29, 245638), 'List the planets in the solar system.', 'planets in solar system', 'Astronomy', '[]', 'The planets are Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune.')
(3, 3, datetime.datetime(2025, 11, 20, 1, 0, 29, 245638), "Translate 'Hello' to Spanish.", 'translate Hello Spanish', 'Language', '[]', "'Hello' in Spanish is 'Hola'.")


In [4]:
print("\nLogs Table Contents:")
try:
    cursor.execute("SELECT id, u_id, timestamp, query, processed_query, context, past_memory, llm_response FROM logs")
    records = cursor.fetchall()
    for record in records:
        print(record)
except Exception as e:
    print(f"✗ Error querying logs table: {e}")


Logs Table Contents:
(1, 1, datetime.datetime(2025, 11, 20, 1, 0, 29, 245638), 'What is the capital of France?', 'capital of France', 'No context', '[]', 'Paris is the capital of France.')
(2, 2, datetime.datetime(2025, 11, 20, 1, 0, 29, 245638), 'List the planets in the solar system.', 'planets in solar system', 'Astronomy', '[]', 'The planets are Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune.')
(3, 3, datetime.datetime(2025, 11, 20, 1, 0, 29, 245638), "Translate 'Hello' to Spanish.", 'translate Hello Spanish', 'Language', '[]', "'Hello' in Spanish is 'Hola'.")
(4, 1, datetime.datetime(2025, 11, 20, 1, 5, 42, 432395), 'What is the coupling reaction?', 'what is the coupling reaction', 'The coupling reaction involves connecting two oligomers using decafluorobiphenyl (10F) at 160°C.', '[]', 'The coupling reaction is a process that connects two oligomers to form diblock copolymers using a termination reagent.')
(5, 1, datetime.datetime(2025, 11, 20, 1, 5, 50, 392304), 'Hi